In [1]:
import pandas as pd
import numpy as np
import vectorbt as vbt

In [2]:
df = pd.read_csv('BTC_4h_binance.csv', index_col=0, parse_dates=True)
df

,Open,High,Low,Close,Volume
timestamp,,,,,
2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865
2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882
2017-08-17 12:00:00,4436.06,4485.39,4333.42,4352.34,174.562001
2017-08-17 16:00:00,4352.33,4354.84,4200.74,4325.23,225.109716
2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08,249.769913
...,...,...,...,...,...
2025-07-17 04:00:00,118128.80,118772.00,117857.03,118311.75,2080.633830
2025-07-17 08:00:00,118311.75,119192.00,117974.99,117995.00,2041.883160
2025-07-17 12:00:00,117995.00,119073.46,117453.57,118881.99,3175.794830


In [5]:
btc = pd.read_csv('BTC_30m_binance.csv', index_col=0, parse_dates=True)
df = btc.copy()

In [29]:
periods = np.arange(20, 3344, 10)

swing_low_df = pd.concat(
    [df['Low'].shift(1).rolling(w, min_periods=1).min() for w in periods],
    axis=1
)
swing_low_df.columns = periods

In [30]:

low_arr = df['Low'].values[:, None]
close_arr = df['Close'].values[:, None]
swing_arr = swing_low_df.values

lower_wick = np.minimum(df['Open'], df['Close']) - df['Low']
upper_wick = df['High'] - np.maximum(df['Open'], df['Close'])
body = np.abs(df['Close'] - df['Open'])

lower_wick_arr = lower_wick.values[:, None]
upper_wick_arr = upper_wick.values[:, None]
body_arr = body.values[:, None]

bullish_arr = (
    (low_arr < swing_arr) &
    (close_arr > swing_arr) #&
    #(lower_wick_arr > upper_wick_arr) &
    #(lower_wick_arr > body_arr)
)

In [31]:
entries = pd.DataFrame(bullish_arr, index=df.index, columns=periods).astype(bool)
exits   = entries.shift(10).fillna(False).astype(bool)

C:\Users\Sexyc\AppData\Local\Temp\ipykernel_21764\1337015734.py:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [32]:

pf = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=entries,
    exits=exits,
    freq='30min',
    init_cash=10_000,
    fees=0.001
)

In [33]:

tr = pf.total_return()
tr.vbt.plot(title='Total Return by Window').show()
print("Total return by window:\n", tr)

best_win = tr.idxmax()
print(f"\nBest‐performing window: {best_win}")

stats = pf.stats()
best_stats = stats[best_win]
print(f"\nStats for window = {best_win}:\n", best_stats)
num_trades = pf.trades.count[best_win]
print(f"\nTotal number of trades for window = {best_win}: {num_trades}")

Total return by window:
 20     -0.994704
30     -0.991196
40     -0.985734
50     -0.967167
60     -0.922665
          ...   
3300   -0.206015
3310   -0.169188
3320   -0.169188
3330   -0.169188
3340   -0.169188
Name: total_return, Length: 333, dtype: float64

Best‐performing window: 1650


C:\Users\Sexyc\AppData\Local\Temp\ipykernel_21764\539607057.py:8: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x000002547F490F40>. Pass column to select a single column/group.

C:\Users\Sexyc\AppData\Local\Temp\ipykernel_21764\539607057.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



IndexError: index 1650 is out of bounds for axis 0 with size 28

In [ ]:
df.to_csv('BTC_4h_binance.csv')